In [1]:

import socket
import mysql.connector
import os


In [2]:
#function to connect with our MySQL database

def connect_to_db ():
    db = mysql.connector.connect(
        host='173.230.133.41',
        user='user',
        password=''
    )

    print ()
    print (db)

    mycursor = db.cursor()
    
    return db, mycursor


In [3]:
#Function to return the local IP and hostname

def get_local_ip():
    try:
        host_name = socket.gethostname()
        host_ip = socket.gethostbyname(host_name)
        print()
        print("Hostname :  ",host_name)
        print("IP : ",host_ip)
    except:
        print("Unable to get Hostname and IP")
        
    return host_ip, host_name


In [4]:
#Function to return all files names in DistShared folder

def get_files():
    
    files = os.listdir('DistShared')
    
    print()
    print("Files From Share Drive:")
    for item in files:
        print(item)
        
    return files


In [5]:
#Function to push all files from DistShared folder to server

def push_to_server ():
    
    db, mycursor = connect_to_db()
    host_ip, host_name = get_local_ip()
    files = get_files()

    print()
    for item in files:
        
        mycursor.execute("USE CDS;")
        
        sql_cmd = "INSERT INTO peer_list (IP, filename) VALUES ("
        iVals = ("'" + host_ip + "', '" + item + "'")
        
        print("Inserted file:   " + item)
        
        mycursor.execute(sql_cmd + iVals + ");")
        
        db.commit()
        

In [6]:
#Function to pull all files down from the server and display for user

def pull_from_server ():
    
    db, mycursor = connect_to_db ()
    
    mycursor.execute("USE CDS")
    mycursor.execute("SELECT * FROM peer_list")
    result = mycursor.fetchall()
    
    print()
    print('Avaliable Files For Download')
    print()
    
    pairlist = []
    count = 0
    for row in result:
        pair = []
        pair = [row[0], row[1]]
        print (pair)
        pairlist.append(pair)
        count = count + 1
        
        print(str(count) + ')   Machine IP Address:   ', row[0] + '      File Name:   ' + row[1])
        
    return pairlist

pull_from_server()



Avaliable Files For Download

['192.168.1.153', 'file4.txt']
1)   Machine IP Address:    192.168.1.153      File Name:   file4.txt
['192.168.1.153', 'file5.txt']
2)   Machine IP Address:    192.168.1.153      File Name:   file5.txt
['192.168.1.153', 'file6.txt']
3)   Machine IP Address:    192.168.1.153      File Name:   file6.txt
['192.168.56.1', 'file1.txt']
4)   Machine IP Address:    192.168.56.1      File Name:   file1.txt
['192.168.56.1', 'file2.txt']
5)   Machine IP Address:    192.168.56.1      File Name:   file2.txt
['192.168.56.1', 'file3.txt']
6)   Machine IP Address:    192.168.56.1      File Name:   file3.txt


[['192.168.1.153', 'file4.txt'],
 ['192.168.1.153', 'file5.txt'],
 ['192.168.1.153', 'file6.txt'],
 ['192.168.56.1', 'file1.txt'],
 ['192.168.56.1', 'file2.txt'],
 ['192.168.56.1', 'file3.txt']]

In [7]:
#Function to delete all files from the server associate with the local IP of current machine

def delete_files ():
    
    db, mycursor = connect_to_db ()
    host_ip, host_name = get_local_ip()
    
    mycursor.execute("USE CDS")
    mycursor.execute("DELETE FROM peer_list WHERE IP = '" + host_ip + "';" )
    
    db.commit()
    
    print ()
    print ("All files associated with this machines IP have been successfully deleted from the server! ")
    

In [8]:
#Main driver function for client

def main():
    
    delete_files()
    push_to_server ()
    pairlist = pull_from_server ()
    
    print ()
    print ("Please enter the index number of the file you wish to download!\n")
    fileindex = int(input())
    
    dest_ip = pairlist [fileindex - 1][0]
    filename = pairlist [fileindex - 1][1]
    
    print ("You are asking for file: " + filename + " from IP address: " + dest_ip + ". Correct? (Y/n)\n")
    proceed = input()
    
    if proceed != 'n' or 'N':
        print("Program terminating...")
        exit()
        
    HOST = dest_ip  # The server's hostname or IP address
    PORT = 44444        # The port used by the server

    with socket.socket(socket.AF_INET, socket.SOCK_STREAM) as s:
        s.connect((HOST, PORT))
        s.sendall(bytes(dest_ip + " " + filename, 'utf-8'))

    print()
    print("Request for file successfully sent!")

if __name__ == "__main__":
    main()
    



Hostname :   DESKTOP-NQAPMAS
IP :  192.168.56.1

All files associated with this machines IP have been successfully deleted from the server! 


Hostname :   DESKTOP-NQAPMAS
IP :  192.168.56.1

Files From Share Drive:
file1.txt
file2.txt
file3.txt

Inserted file:   file1.txt
Inserted file:   file2.txt
Inserted file:   file3.txt


Avaliable Files For Download

['192.168.1.153', 'file4.txt']
1)   Machine IP Address:    192.168.1.153      File Name:   file4.txt
['192.168.1.153', 'file5.txt']
2)   Machine IP Address:    192.168.1.153      File Name:   file5.txt
['192.168.1.153', 'file6.txt']
3)   Machine IP Address:    192.168.1.153      File Name:   file6.txt
['192.168.56.1', 'file1.txt']
4)   Machine IP Address:    192.168.56.1      File Name:   file1.txt
['192.168.56.1', 'file2.txt']
5)   Machine IP Address:    192.168.56.1      File Name:   file2.txt
['192.168.56.1', 'file3.txt']
6)   Machine IP Address:    192.168.56.1      File Name:   file3.txt

Please enter the index number of the 